# Using `catboost.metrics` module

In this short tutorial, we'll show you the benefits of using the metrics from `catboost.metrics`.

`catboost.metrics` module contains classes for all metrics (or loss functions) available for use with Catboost. By importing it, you can view the list of metrics, their supported parameters and their default values. Formulas and descriptions for the metrics are available in documentation: https://catboost.ai/docs/concepts/loss-functions.html.

In [1]:
from catboost import metrics

In [2]:
dir(metrics)

['AUC',
 'AUCMulticlass',
 'Accuracy',
 'AverageGain',
 'BalancedAccuracy',
 'BalancedErrorRate',
 'BrierScore',
 'BuiltinMetric',
 'Combination',
 'CrossEntropy',
 'CtrFactor',
 'DCG',
 'DataFrame',
 'ERR',
 'Expectile',
 'F1',
 'FairLoss',
 'FilteredDCG',
 'HammingLoss',
 'HingeLoss',
 'Huber',
 'Kappa',
 'LogLikelihoodOfPrediction',
 'LogLinQuantile',
 'Logloss',
 'Lq',
 'MAE',
 'MAP',
 'MAPE',
 'MCC',
 'MRR',
 'MSLE',
 'MedianAbsoluteError',
 'MultiClass',
 'MultiClassOneVsAll',
 'MultiRMSE',
 'NDCG',
 'NormalizedGini',
 'NormalizedGiniMulticlass',
 'NumErrors',
 'PFound',
 'PRAUC',
 'PRAUCMulticlass',
 'PairAccuracy',
 'PairLogit',
 'Poisson',
 'Precision',
 'PrecisionAt',
 'Quantile',
 'QueryAUC',
 'QueryAUCMulticlass',
 'QueryAverage',
 'QueryCrossEntropy',
 'QueryRMSE',
 'QuerySoftMax',
 'R2',
 'RMSE',
 'RMSEWithUncertainty',
 'Recall',
 'RecallAt',
 'SMAPE',
 'Series',
 'TotalF1',
 'Tweedie',
 'UserQuerywiseMetric',
 'WKappa',
 'ZeroOneLoss',
 '_ARRAY_TYPES',
 '_MetricGenerato

Now you can inspect the metrics: for example, `Lq` has one mandatory parameter `q` and parameters `use_weights` with default value `True` and no `hints`:

In [3]:
metrics.Lq

Builtin metric: 'Lq'
Parameters:
    q (mandatory)
    use_weights = True (default value)
    hints = '' (default value)

In contrast, `AUC` metric has no mandatory parameters, and its computation is disabled on train by default:

In [4]:
metrics.AUC

Builtin metric: 'AUC'
Parameters:
    type = 'Classic' (default value)
    hints = 'skip_train~true' (default value)
    use_weights = False (default value)

To use the metric, you can first create it by supplying the required parameters. The constructor will warn you if you forget something or make a typo:

In [5]:
m = metrics.Lq()

ValueError: Parameter q is mandatory and must be specified.

In [6]:
m = metrics.Lq(Q=4)

ValueError: Unexpected parameter Q

In [7]:
# successful creation of a metric:
m = metrics.Lq(q=4)
m

Lq(q=4 [mandatory=True], use_weights=True [mandatory=False])

Metric classes have some helper methods, such as:
* `set_hints(**kwargs)` – for setting hints,
* `params_with_defaults()` – for programmatic access to parameter information,
* `eval(labels, approxes, <other params: weight, group_id, group_weight, subgroup_id, pairs, thread_count>)` – to evaluate the metric on given true labels and predicted labels.

In [8]:
m.set_hints(skip_train=True)
m

Lq(hints='skip_train~true' [mandatory=False], q=4 [mandatory=True], use_weights=True [mandatory=False])

In [9]:
m.q

4

In [10]:
m.params_with_defaults()

{'q': {'default_value': None, 'is_mandatory': True},
 'use_weights': {'default_value': True, 'is_mandatory': False},
 'hints': {'default_value': '', 'is_mandatory': False}}

We can now use this metric in training or evaluation:

In [11]:
from catboost import CatBoostRegressor

# Initialize toy data
train_data = [[1, 4, 5, 6],
              [4, 5, 6, 7],
              [30, 40, 50, 60]]

eval_data = [[2, 4, 6, 8],
             [1, 4, 50, 60]]

train_labels = [10, 20, 30]

eval_labels = [12, 15]

# Initialize CatBoostRegressor
model = CatBoostRegressor(
    iterations=5,
    learning_rate=1,
    depth=2,
    loss_function=m,
    custom_metric=[metrics.Lq(q=5), metrics.R2()]
)

# Fit model
model.fit(train_data, train_labels)

0:	total: 50.8ms	remaining: 203ms
1:	total: 51.5ms	remaining: 77.3ms
2:	total: 51.9ms	remaining: 34.6ms
3:	total: 52.3ms	remaining: 13.1ms
4:	total: 52.8ms	remaining: 0us


In [12]:
print(model.get_best_score())

{'learn': {'Lq:q=5': 388.4841340753533, 'R2': 0.8778546529631635}}


In [13]:
# Get predictions
preds = model.predict(eval_data)

In [14]:
metrics.RMSE().eval(eval_labels, preds)

[5.107940380990622]